## BEST SCORES 

Learning rates - 1e-1, 1e-2, 1e-3, 1e-4  <br>
Hidden layers - 1, 3  <br>

Hidden layer 3 had all scores around 45-50 and it did not increase after 50 epochs as well. Hidden layer 1 had decent scores for Learning rates 1e-3, 1e-4  <br>

#### RNN
1e-4 with 1 hidden layer, epoch 48 - 57 <br>
Best -  1e-3 with 1 hidden layer, epoch 58 - 63  <br>

#### Gated RNN
Best -  1e-3 with 1 hidden layer, epoch 51 - 76  <br>

#### LSTM 
1e-3 with 2/3 hidden layer, epoch 8 - 52  <br>
Stopped as it started going down and reached 0  <br>

Best - 1e-3 and 1 hidden layer, epoch 48 - 65  <br>

#### Bi LSTM
Best - 1e-3 and 1 hidden layer, epoch 45 - 65  <br>

In [1]:
# dependencies to run the notebook

!pip install torch==1.12.1
!pip install torchmetrics==0.10.2
!pip install torchvision==0.14.0
!pip install texttable==1.6.4


<span style="color:darkviolet">
<font size="3">Download the below files from https://drive.google.com/drive/folders/1q50QMurzK9a5l4JBHWjf8VuWcZkbF7PM to run this notebook : <br>
1) train_bert_embeddings.pkl <br>
2) test_bert_embeddings.pkl <br> </font>
</span>


In [1]:
import pickle
import torch
import torch.nn as nn
from torch.utils.data import DataLoader, TensorDataset
from torch.utils.data import Dataset
from torchvision.transforms import ToTensor
from torch.autograd import Variable
import torch.optim as optim
import warnings
import numpy as np
import torch.nn.functional as F
import pandas as pd
from sklearn.metrics import classification_report
from texttable import Texttable
from torch.utils.data.sampler import SubsetRandomSampler

warnings.filterwarnings("ignore")


/Library/Frameworks/Python.framework/Versions/3.8/lib/python3.8/site-packages/torchvision/io/image.py:13: UserWarning: Failed to load image Python extension: dlopen(/Library/Frameworks/Python.framework/Versions/3.8/lib/python3.8/site-packages/torchvision/image.so, 0x0006): Symbol not found: __ZN2at4_ops19empty_memory_format4callEN3c108ArrayRefINS2_6SymIntEEENS2_8optionalINS2_10ScalarTypeEEENS6_INS2_6LayoutEEENS6_INS2_6DeviceEEENS6_IbEENS6_INS2_12MemoryFormatEEE
  Referenced from: /Library/Frameworks/Python.framework/Versions/3.8/lib/python3.8/site-packages/torchvision/image.so
  Expected in: /Library/Frameworks/Python.framework/Versions/3.8/lib/python3.8/site-packages/torch/lib/libtorch_cpu.dylib
  warn(f"Failed to load image Python extension: {e}")


In [2]:
# Read train BERT embeddings
with open("train_roberta_embeddings.pkl", "rb") as f:
    training_data = pickle.load(f)


In [3]:
# Read test BERT embeddings
with open("test_roberta_embeddings.pkl", "rb") as f:
    testing_data = pickle.load(f)


In [4]:
# check
for item in training_data:
    print("The data is : ", item)


The data is :  embeddings
The data is :  tokenized_txt


In [5]:
len(training_data["embeddings"]), training_data["embeddings"][0].shape


(7531, torch.Size([1, 512, 768]))

In [6]:
len(training_data["tokenized_txt"]), len(training_data["tokenized_txt"][0])


(7531, 512)

<span style="color:darkviolet">
<font size="4">Get the labels from train and test files.</font>
</span>


In [7]:
train_dataset_path = "tos_clauses_train.csv"
test_dataset_path = "tos_clauses_dev.csv"


In [8]:
train_df = pd.read_csv(train_dataset_path, header=0)
test_df = pd.read_csv(test_dataset_path, header=0)


In [9]:
train_targets = train_df.label.values
test_targets = test_df.label.values


In [10]:
test_targets


array([0, 0, 0, ..., 0, 1, 0])

In [11]:
device = None
if torch.cuda.is_available():
    device = torch.device("cuda")
# elif torch.backends.mps.is_available():
#     device = torch.device("mps")
else:
    device = torch.device("cpu")

print(f"Using Device: {device}")


Using Device: cpu


<span style="color:darkviolet">
<font size="4">Create Dataset, Train and Test Classes</font>
</span>


In [12]:
class Dataset(object):
    """An abstract class representing a Dataset.
    All other datasets should subclass it. All subclasses should
    override ``__len__``, that provides the size of the dataset,
    and ``__getitem__``, supporting integer indexing in range
    from 0 to len(self) exclusive.
    """

    def __getitem__(self, index):
        raise NotImplementedError

    def __len__(self):
        raise NotImplementedError

    def __add__(self, other):
        return ConcatDataset([self, other])


In [13]:
class TOSDataset(Dataset):
    def __init__(self, X, Y):
        self.data1 = X
        self.data2 = Y

    def __len__(self):
        return len(self.data1)

    def __getitem__(self, index):
        x = self.data1[index]
        y = self.data2[index]

        x = torch.tensor(x)

        return torch.squeeze(x, dim=1), torch.tensor(y)


In [14]:
test_len = len(test_df)
train_len = len(train_df)
X_train_tensor = TOSDataset(train_df["sentences"], train_df["label"])
# X_test_tensor = Train_Model(test_df)

num_train = len(X_train_tensor)
indices = list(range(num_train))
np.random.shuffle(indices)
# split = int(np.floor(num_train))
# train_idx = indices[split:]

train_sampler = SubsetRandomSampler(indices)
# valid_sampler = SubsetRandomSampler(valid_idx)
print(train_sampler)
train_df_by_index = train_df.loc[indices]
# val_df_by_index = df_train.loc[valid_idx]
train_fair = sum(train_df_by_index["label"] == 0)
train_unfair = sum(train_df_by_index["label"] == 1)
# val_fair = sum(val_df_by_index['label'] == 0)
# val_unfair = sum(val_df_by_index['label'] == 1)

print("train_fair:" + str(train_fair))
print("train_unfair:" + str(train_unfair))
# print("val_fair:" + str(val_fair))
# print("val_unfair:" + str(val_unfair))


train_fair:6705
train_unfair:826


In [15]:
train_data = TOSDataset(training_data["embeddings"], train_targets)
test_data = TOSDataset(testing_data["embeddings"], test_targets)


<span style="color:darkviolet">
<font size="4">Prepare Data loaders</font>
</span>


In [16]:
# how many samples per batch to load
BATCH_SIZE = 20

# number of subprocesses to use for data loading
NUM_WORKERS = 0


In [17]:
# prepare data loaders
train_loader = DataLoader(
    train_data, batch_size=BATCH_SIZE, sampler=train_sampler, num_workers=NUM_WORKERS
)
test_loader = DataLoader(
    test_data, batch_size=BATCH_SIZE, num_workers=NUM_WORKERS
)


In [18]:
# check sizes
dataiter = iter(train_loader)
sample_x, sample_y = dataiter.next()

print("Sample input size: ", sample_x.size())  # batch_size, seq_length
# print("Sample input: \n", sample_x)
print()
print("Sample label size: ", sample_y.size())  # batch_size
# print("Sample label: \n", sample_y)


Sample input size:  torch.Size([20, 1, 512, 768])

Sample label size:  torch.Size([20])


In [19]:
torch.squeeze(sample_x, dim=1).shape


torch.Size([20, 512, 768])

<span style="color:darkviolet">
<font size="5">SIMPLE RNN</font><br>
<font size="2.5">Number of hidden dimension : 20</font> <br>
<font size="2.5">Number of layers: 1</font> <br>
<font size="2.5">Number of epochs: 5</font> <br>
</span>


In [20]:
class RNNet(nn.Module):
    def __init__(self, input_dim, hidden_dim, output_dim):

        super(RNNet, self).__init__()

        # Number of hidden dimensions
        self.hidden_dim = hidden_dim

        # RNN
        self.rnn = nn.RNN(input_dim, hidden_dim, num_layers=1, batch_first=True, nonlinearity="relu")

        # Readout layer
        self.fc = nn.Linear(hidden_dim, output_dim)

    def forward(self, x):

        # Initialize hidden state with zeros
        h0 = Variable(torch.zeros(1, x.size(0), self.hidden_dim))

        # One time step
        out, hn = self.rnn(x, h0)
        out = self.fc(out[:, -1, :])
        return out


In [20]:
import time


def epoch_time(start_time, end_time):
    elapsed_time = end_time - start_time
    elapsed_mins = int(elapsed_time / 60)
    elapsed_secs = int(elapsed_time - (elapsed_mins * 60))
    return elapsed_mins, elapsed_secs


In [21]:
EMBEDDING_DIM = 768
HIDDEN_DIM = 512
OUTPUT_DIM = 2
N_EPOCHS = 75



In [22]:
from sklearn.utils.class_weight import compute_class_weight

class_weight = compute_class_weight(
    "balanced", classes=np.unique(train_df_by_index["label"]), y=train_df_by_index["label"]
)
class_weight


array([0.56159582, 4.55871671])

In [24]:
model = RNNet(EMBEDDING_DIM, HIDDEN_DIM, OUTPUT_DIM)

loss_fn = nn.CrossEntropyLoss(weight=torch.FloatTensor(class_weight))
optimizer = optim.Adam(model.parameters(), lr=1e-3)
test_min_loss = np.inf

for epoch in range(N_EPOCHS):

    start_time = time.time()
    model.train()
    train_loss = 0.0
    test_loss = 0.0
    for inputs, target in train_loader:
        model.zero_grad()
        inputs = torch.squeeze(inputs, dim=1)
        output = model(inputs)
        loss = loss_fn(output, target)
        loss.backward()
        train_loss += loss.item()
        optimizer.step()

    y_pred_list = []
    y_targ_list = []
    model.eval()
    for inputs, target in test_loader:
        inputs = torch.squeeze(inputs, dim=1)
        output = model(inputs)
        loss = loss_fn(output, target)
        test_loss += loss.item()
        _, y_test_pred = torch.max(output, 1)
        y_pred_tag = y_test_pred
        y_pred_list.append(y_pred_tag.cpu().numpy())
        y_targ_list.append(target.cpu().numpy())
    

    if(epoch%1 == 0):
        y_pred_list = [x.squeeze().tolist() for x in y_pred_list]
        y_targ_list = [x.squeeze().tolist() for x in y_targ_list]
        y_pred_list = [x for sublist in y_pred_list for x in sublist]
        y_targ_list = [x for sublist in y_targ_list for x in sublist]

        print(classification_report(y_targ_list, y_pred_list))

    train_loss = train_loss / len(train_loader.dataset)
    test_loss = test_loss / len(test_loader.dataset)

    end_time = time.time()

    epoch_mins, epoch_secs = epoch_time(start_time, end_time)

    print(f"Epoch: {epoch+1:02} | Epoch Time: {epoch_mins}m {epoch_secs}s")
    print("\tTraining Loss: {:.6f} \Test Loss: {:.6f}".format(train_loss, test_loss))
    if test_loss <= test_min_loss:
        print("Test loss decreased ({:.6f} --> {:.6f}). Saving model...".format(test_min_loss, test_loss))
        torch.save(model.state_dict(), "models_rnn_roberta/rnn_bert_model_1_3.pt")
        test_min_loss = test_loss


              precision    recall  f1-score   support

           0       0.93      0.82      0.87      1677
           1       0.26      0.50      0.34       206

    accuracy                           0.79      1883
   macro avg       0.60      0.66      0.61      1883
weighted avg       0.86      0.79      0.82      1883

Epoch: 01 | Epoch Time: 1m 2s
	Training Loss: 0.034089 \Test Loss: 0.032139
Test loss decreased (inf --> 0.032139). Saving model...
              precision    recall  f1-score   support

           0       0.94      0.80      0.87      1677
           1       0.28      0.62      0.38       206

    accuracy                           0.78      1883
   macro avg       0.61      0.71      0.62      1883
weighted avg       0.87      0.78      0.81      1883

Epoch: 02 | Epoch Time: 1m 1s
	Training Loss: 0.030792 \Test Loss: 0.028430
Test loss decreased (0.032139 --> 0.028430). Saving model...
              precision    recall  f1-score   support

           0       0.9

              precision    recall  f1-score   support

           0       0.99      0.68      0.81      1677
           1       0.26      0.92      0.41       206

    accuracy                           0.71      1883
   macro avg       0.62      0.80      0.61      1883
weighted avg       0.91      0.71      0.76      1883

Epoch: 20 | Epoch Time: 1m 9s
	Training Loss: 0.017191 \Test Loss: 0.023261
              precision    recall  f1-score   support

           0       0.97      0.87      0.92      1677
           1       0.43      0.78      0.55       206

    accuracy                           0.86      1883
   macro avg       0.70      0.82      0.74      1883
weighted avg       0.91      0.86      0.88      1883

Epoch: 21 | Epoch Time: 1m 9s
	Training Loss: 0.016744 \Test Loss: 0.018985
              precision    recall  f1-score   support

           0       0.98      0.82      0.89      1677
           1       0.37      0.86      0.52       206

    accuracy                  

              precision    recall  f1-score   support

           0       0.97      0.90      0.94      1677
           1       0.50      0.77      0.60       206

    accuracy                           0.89      1883
   macro avg       0.73      0.84      0.77      1883
weighted avg       0.92      0.89      0.90      1883

Epoch: 40 | Epoch Time: 1m 11s
	Training Loss: 0.014501 \Test Loss: 0.018890
              precision    recall  f1-score   support

           0       0.99      0.79      0.88      1677
           1       0.35      0.91      0.50       206

    accuracy                           0.80      1883
   macro avg       0.67      0.85      0.69      1883
weighted avg       0.92      0.80      0.84      1883

Epoch: 41 | Epoch Time: 1m 10s
	Training Loss: 0.014599 \Test Loss: 0.019250
              precision    recall  f1-score   support

           0       0.98      0.82      0.89      1677
           1       0.37      0.87      0.52       206

    accuracy                

              precision    recall  f1-score   support

           0       0.97      0.90      0.93      1677
           1       0.49      0.79      0.61       206

    accuracy                           0.89      1883
   macro avg       0.73      0.85      0.77      1883
weighted avg       0.92      0.89      0.90      1883

Epoch: 61 | Epoch Time: 1m 0s
	Training Loss: 0.013079 \Test Loss: 0.018156
              precision    recall  f1-score   support

           0       0.99      0.79      0.88      1677
           1       0.35      0.91      0.51       206

    accuracy                           0.81      1883
   macro avg       0.67      0.85      0.69      1883
weighted avg       0.92      0.81      0.84      1883

Epoch: 62 | Epoch Time: 1m 0s
	Training Loss: 0.012634 \Test Loss: 0.019204
              precision    recall  f1-score   support

           0       0.95      0.94      0.95      1677
           1       0.58      0.62      0.60       206

    accuracy                  

<span style="color:darkviolet">
<font size="5">Gated RNN</font><br>
</span>


In [23]:
class GRU_Network(nn.Module):
    def __init__(self, input_dim, hidden_dim, output_dim):

        super(GRU_Network, self).__init__()

        # Number of hidden dimensions
        self.hidden_dim = hidden_dim

        # RNN
        self.rnn = nn.GRU(input_dim, hidden_dim, num_layers=1, batch_first=True)

        # Readout layer
        self.fc = nn.Linear(hidden_dim, output_dim)

    def forward(self, x):

        # Initialize hidden state with zeros
        h0 = Variable(torch.zeros(1, x.size(0), self.hidden_dim))

        # One time step
        out, hn = self.rnn(x, h0)
        out = self.fc(out[:, -1, :])
        return out


In [24]:
EMBEDDING_DIM = 768
HIDDEN_DIM = 512
OUTPUT_DIM = 2
N_EPOCHS = 100

In [25]:
model_gru = GRU_Network(EMBEDDING_DIM, HIDDEN_DIM, OUTPUT_DIM)
print(model_gru)
test_min_loss = np.inf
criterion = nn.CrossEntropyLoss(weight=torch.FloatTensor(class_weight))
optimizer_gru = optim.Adam(model_gru.parameters(), lr=1e-3)
    
for epoch in range(N_EPOCHS):
    start_time = time.time()
    model_gru.train()
    train_loss = 0.0
    test_loss = 0.0
    for inputs, target in train_loader:
        model_gru.zero_grad()
        inputs = torch.squeeze(inputs, dim=1)
        output = model_gru(inputs)
        loss = criterion(output, target)
        loss.backward()
        train_loss += loss.item()
        optimizer_gru.step()

    y_pred_list = []
    y_targ_list = []
    model_gru.eval()
    for inputs, target in test_loader:
        inputs = torch.squeeze(inputs, dim=1)
        output = model_gru(inputs)
        loss = criterion(output, target)
        test_loss += loss.item()
        _, y_test_pred = torch.max(output, 1)
        y_pred_tag = y_test_pred
        y_pred_list.append(y_pred_tag.cpu().numpy())
        y_targ_list.append(target.cpu().numpy())
    

    if(epoch%1 == 0):
        y_pred_list = [x.squeeze().tolist() for x in y_pred_list]
        y_targ_list = [x.squeeze().tolist() for x in y_targ_list]
        y_pred_list = [x for sublist in y_pred_list for x in sublist]
        y_targ_list = [x for sublist in y_targ_list for x in sublist]

        print(classification_report(y_targ_list, y_pred_list))

    train_loss = train_loss / len(train_loader.dataset)
    test_loss = test_loss / len(test_loader.dataset)

    end_time = time.time()

    epoch_mins, epoch_secs = epoch_time(start_time, end_time)

    print(f"Epoch: {epoch+1:02} | Epoch Time: {epoch_mins}m {epoch_secs}s")
    print("\tTraining Loss: {:.6f} \Test Loss: {:.6f}".format(train_loss, test_loss))
    if test_loss <= test_min_loss:
        print("Test loss decreased ({:.6f} --> {:.6f}). Saving model...".format(test_min_loss, test_loss))
        torch.save(model_gru.state_dict(), "models_gru_roberta/gru_bert_model_1_3.pt")
        test_min_loss = test_loss


GRU_Network(
  (rnn): GRU(768, 512, batch_first=True)
  (fc): Linear(in_features=512, out_features=2, bias=True)
)
              precision    recall  f1-score   support

           0       0.98      0.13      0.23      1677
           1       0.12      0.98      0.22       206

    accuracy                           0.22      1883
   macro avg       0.55      0.56      0.22      1883
weighted avg       0.89      0.22      0.23      1883

Epoch: 01 | Epoch Time: 3m 33s
	Training Loss: 0.032791 \Test Loss: 0.038170
Test loss decreased (inf --> 0.038170). Saving model...
              precision    recall  f1-score   support

           0       0.95      0.86      0.90      1677
           1       0.35      0.62      0.45       206

    accuracy                           0.83      1883
   macro avg       0.65      0.74      0.68      1883
weighted avg       0.88      0.83      0.85      1883

Epoch: 02 | Epoch Time: 3m 27s
	Training Loss: 0.027904 \Test Loss: 0.024995
Test loss decreased (

              precision    recall  f1-score   support

           0       0.97      0.97      0.97      1677
           1       0.74      0.73      0.73       206

    accuracy                           0.94      1883
   macro avg       0.85      0.85      0.85      1883
weighted avg       0.94      0.94      0.94      1883

Epoch: 20 | Epoch Time: 3m 28s
	Training Loss: 0.001983 \Test Loss: 0.032882
              precision    recall  f1-score   support

           0       0.97      0.97      0.97      1677
           1       0.74      0.72      0.73       206

    accuracy                           0.94      1883
   macro avg       0.85      0.84      0.85      1883
weighted avg       0.94      0.94      0.94      1883

Epoch: 21 | Epoch Time: 3m 25s
	Training Loss: 0.000808 \Test Loss: 0.038451
              precision    recall  f1-score   support

           0       0.97      0.98      0.97      1677
           1       0.80      0.72      0.76       206

    accuracy                

              precision    recall  f1-score   support

           0       0.96      0.98      0.97      1677
           1       0.81      0.71      0.75       206

    accuracy                           0.95      1883
   macro avg       0.89      0.84      0.86      1883
weighted avg       0.95      0.95      0.95      1883

Epoch: 41 | Epoch Time: 3m 49s
	Training Loss: 0.000023 \Test Loss: 0.061413
              precision    recall  f1-score   support

           0       0.96      0.98      0.97      1677
           1       0.81      0.71      0.75       206

    accuracy                           0.95      1883
   macro avg       0.89      0.84      0.86      1883
weighted avg       0.95      0.95      0.95      1883

Epoch: 42 | Epoch Time: 3m 30s
	Training Loss: 0.000015 \Test Loss: 0.060986
              precision    recall  f1-score   support

           0       0.96      0.98      0.97      1677
           1       0.83      0.66      0.73       206

    accuracy                

[E thread_pool.cpp:113] Exception in thread pool task: mutex lock failed: Invalid argument


KeyboardInterrupt: 

<span style="color:darkviolet">
<font size="5">LSTM</font><br>
</span>


In [25]:
# LSTM


class LSTM_Network(nn.Module):
    def __init__(self, input_dim, hidden_dim, output_dim):

        super(LSTM_Network, self).__init__()

        # Number of hidden dimensions
        self.hidden_dim = hidden_dim

        # RNN
        self.lstm = nn.LSTM(input_dim, hidden_dim, num_layers=1, batch_first=True)

        # Readout layer
        self.fc = nn.Linear(hidden_dim, output_dim)

    def forward(self, x):

        # Initialize hidden state with zeros
        h0 = Variable(torch.zeros(1, x.size(0), self.hidden_dim))
        c0 = Variable(torch.zeros(1, x.size(0), self.hidden_dim))

        # One time step
        out, (hn, cn) = self.lstm(x, (h0, c0))
        out = self.fc(out[:, -1, :])

        return out


In [26]:
model_lstm = LSTM_Network(EMBEDDING_DIM, HIDDEN_DIM, OUTPUT_DIM)
print(model_lstm)

test_min_loss = np.inf
criterion = nn.CrossEntropyLoss(weight=torch.FloatTensor(class_weight))
optimizer = optim.Adam(model_lstm.parameters(), lr=1e-3)
    
for epoch in range(N_EPOCHS):
    start_time = time.time()
    model_lstm.train()
    train_loss = 0.0
    test_loss = 0.0
    for inputs, target in train_loader:
        model_lstm.zero_grad()
        inputs = torch.squeeze(inputs, dim=1)
        output = model_lstm(inputs)
        loss = criterion(output, target)
        loss.backward()
        train_loss += loss.item()
        optimizer.step()

    y_pred_list = []
    y_targ_list = []
    model_lstm.eval()
    for inputs, target in test_loader:
        inputs = torch.squeeze(inputs, dim=1)
        output = model_lstm(inputs)
        loss = criterion(output, target)
        test_loss += loss.item()
        _, y_test_pred = torch.max(output, 1)
        y_pred_tag = y_test_pred
        y_pred_list.append(y_pred_tag.cpu().numpy())
        y_targ_list.append(target.cpu().numpy())
    

    if(epoch%1 == 0):
        y_pred_list = [x.squeeze().tolist() for x in y_pred_list]
        y_targ_list = [x.squeeze().tolist() for x in y_targ_list]
        y_pred_list = [x for sublist in y_pred_list for x in sublist]
        y_targ_list = [x for sublist in y_targ_list for x in sublist]

        print(classification_report(y_targ_list, y_pred_list))

    train_loss = train_loss / len(train_loader.dataset)
    test_loss = test_loss / len(test_loader.dataset)

    end_time = time.time()

    epoch_mins, epoch_secs = epoch_time(start_time, end_time)

    print(f"Epoch: {epoch+1:02} | Epoch Time: {epoch_mins}m {epoch_secs}s")
    print("\tTraining Loss: {:.6f} \Test Loss: {:.6f}".format(train_loss, test_loss))
    if test_loss <= test_min_loss:
        print("Test loss decreased ({:.6f} --> {:.6f}). Saving model...".format(test_min_loss, test_loss))
        torch.save(model_lstm.state_dict(), "models_lstm_roberta/lstm_bert_model_1_3.pt")
        test_min_loss = test_loss


LSTM_Network(
  (lstm): LSTM(768, 512, batch_first=True)
  (fc): Linear(in_features=512, out_features=2, bias=True)
)
              precision    recall  f1-score   support

           0       0.97      0.43      0.59      1677
           1       0.16      0.89      0.27       206

    accuracy                           0.48      1883
   macro avg       0.57      0.66      0.43      1883
weighted avg       0.88      0.48      0.56      1883

Epoch: 01 | Epoch Time: 5m 34s
	Training Loss: 0.033429 \Test Loss: 0.031960
Test loss decreased (inf --> 0.031960). Saving model...
              precision    recall  f1-score   support

           0       0.97      0.62      0.75      1677
           1       0.22      0.87      0.35       206

    accuracy                           0.64      1883
   macro avg       0.60      0.74      0.55      1883
weighted avg       0.89      0.64      0.71      1883

Epoch: 02 | Epoch Time: 5m 18s
	Training Loss: 0.029390 \Test Loss: 0.027660
Test loss decrease

              precision    recall  f1-score   support

           0       0.96      0.93      0.95      1677
           1       0.55      0.67      0.60       206

    accuracy                           0.90      1883
   macro avg       0.76      0.80      0.77      1883
weighted avg       0.91      0.90      0.91      1883

Epoch: 20 | Epoch Time: 5m 14s
	Training Loss: 0.016358 \Test Loss: 0.021525
              precision    recall  f1-score   support

           0       0.94      0.96      0.95      1677
           1       0.64      0.53      0.58       206

    accuracy                           0.92      1883
   macro avg       0.79      0.75      0.77      1883
weighted avg       0.91      0.92      0.91      1883

Epoch: 21 | Epoch Time: 5m 0s
	Training Loss: 0.016672 \Test Loss: 0.026087
              precision    recall  f1-score   support

           0       0.97      0.87      0.92      1677
           1       0.43      0.79      0.56       206

    accuracy                 

              precision    recall  f1-score   support

           0       0.96      0.93      0.94      1677
           1       0.54      0.68      0.60       206

    accuracy                           0.90      1883
   macro avg       0.75      0.80      0.77      1883
weighted avg       0.91      0.90      0.91      1883

Epoch: 40 | Epoch Time: 4m 58s
	Training Loss: 0.013047 \Test Loss: 0.020871
              precision    recall  f1-score   support

           0       0.97      0.92      0.94      1677
           1       0.53      0.74      0.62       206

    accuracy                           0.90      1883
   macro avg       0.75      0.83      0.78      1883
weighted avg       0.92      0.90      0.91      1883

Epoch: 41 | Epoch Time: 4m 52s
	Training Loss: 0.013374 \Test Loss: 0.018890
              precision    recall  f1-score   support

           0       0.96      0.94      0.95      1677
           1       0.57      0.69      0.62       206

    accuracy                

<span style="color:darkviolet">
<font size="5">Bi-LSTM</font><br>
</span>


In [23]:
class Bi_LSTM_Network(nn.Module):
    def __init__(self, input_dim, hidden_dim, output_dim):

        super(Bi_LSTM_Network, self).__init__()

        # Number of hidden dimensions
        self.hidden_dim = hidden_dim

        # RNN
        self.lstm = nn.LSTM(input_dim, hidden_dim, num_layers=1, batch_first=True, bidirectional=True)

        # Readout layer
        self.fc = nn.Linear(hidden_dim * 2, output_dim)

    def forward(self, x):

        # Initialize hidden state with zeros
        h0 = Variable(torch.zeros(1 * 2, x.size(0), self.hidden_dim))
        c0 = Variable(torch.zeros(1 * 2, x.size(0), self.hidden_dim))

        # One time step
        out, (hn, cn) = self.lstm(x, (h0, c0))
        out = self.fc(out[:, -1, :])

        return out


In [ ]:
N_EPOCHS = 75
model_bi_lstm = Bi_LSTM_Network(EMBEDDING_DIM, HIDDEN_DIM, OUTPUT_DIM)
print(model_bi_lstm)

test_min_loss = np.inf
criterion = nn.CrossEntropyLoss(weight=torch.FloatTensor(class_weight))
optimizer = optim.Adam(model_bi_lstm.parameters(), lr=1e-3)
    
for epoch in range(N_EPOCHS):
    start_time = time.time()
    model_bi_lstm.train()
    train_loss = 0.0
    test_loss = 0.0
    for inputs, target in train_loader:
        model_bi_lstm.zero_grad()
        inputs = torch.squeeze(inputs, dim=1)
        output = model_bi_lstm(inputs)
        loss = criterion(output, target)
        loss.backward()
        train_loss += loss.item()
        optimizer.step()

    y_pred_list = []
    y_targ_list = []
    model_bi_lstm.eval()
    for inputs, target in test_loader:
        inputs = torch.squeeze(inputs, dim=1)
        output = model_bi_lstm(inputs)
        loss = criterion(output, target)
        test_loss += loss.item()
        _, y_test_pred = torch.max(output, 1)
        y_pred_tag = y_test_pred
        y_pred_list.append(y_pred_tag.cpu().numpy())
        y_targ_list.append(target.cpu().numpy())
    

    if(epoch%1 == 0):
        y_pred_list = [x.squeeze().tolist() for x in y_pred_list]
        y_targ_list = [x.squeeze().tolist() for x in y_targ_list]
        y_pred_list = [x for sublist in y_pred_list for x in sublist]
        y_targ_list = [x for sublist in y_targ_list for x in sublist]

        print(classification_report(y_targ_list, y_pred_list))

    train_loss = train_loss / len(train_loader.dataset)
    test_loss = test_loss / len(test_loader.dataset)

    end_time = time.time()

    epoch_mins, epoch_secs = epoch_time(start_time, end_time)

    print(f"Epoch: {epoch+1:02} | Epoch Time: {epoch_mins}m {epoch_secs}s")
    print("\tTraining Loss: {:.6f} \Test Loss: {:.6f}".format(train_loss, test_loss))
    if test_loss <= test_min_loss:
        print("Test loss decreased ({:.6f} --> {:.6f}). Saving model...".format(test_min_loss, test_loss))
        torch.save(model_bi_lstm.state_dict(), "models_lstm_roberta/bilstm_bert_model_1_3.pt")
        test_min_loss = test_loss



Bi_LSTM_Network(
  (lstm): LSTM(768, 512, batch_first=True, bidirectional=True)
  (fc): Linear(in_features=1024, out_features=2, bias=True)
)
              precision    recall  f1-score   support

           0       0.93      0.89      0.91      1677
           1       0.34      0.49      0.40       206

    accuracy                           0.84      1883
   macro avg       0.64      0.69      0.66      1883
weighted avg       0.87      0.84      0.85      1883

Epoch: 01 | Epoch Time: 11m 53s
	Training Loss: 0.032383 \Test Loss: 0.028116
Test loss decreased (inf --> 0.028116). Saving model...
              precision    recall  f1-score   support

           0       0.92      0.94      0.93      1677
           1       0.44      0.38      0.41       206

    accuracy                           0.88      1883
   macro avg       0.68      0.66      0.67      1883
weighted avg       0.87      0.88      0.88      1883

Epoch: 02 | Epoch Time: 12m 28s
	Training Loss: 0.027589 \Test Loss: 0

              precision    recall  f1-score   support

           0       0.95      0.93      0.94      1677
           1       0.54      0.64      0.59       206

    accuracy                           0.90      1883
   macro avg       0.75      0.79      0.76      1883
weighted avg       0.91      0.90      0.90      1883

Epoch: 20 | Epoch Time: 11m 15s
	Training Loss: 0.016337 \Test Loss: 0.020810
              precision    recall  f1-score   support

           0       0.98      0.80      0.88      1677
           1       0.35      0.86      0.50       206

    accuracy                           0.81      1883
   macro avg       0.66      0.83      0.69      1883
weighted avg       0.91      0.81      0.84      1883

Epoch: 21 | Epoch Time: 12m 17s
	Training Loss: 0.016335 \Test Loss: 0.018521
              precision    recall  f1-score   support

           0       0.96      0.92      0.94      1677
           1       0.52      0.72      0.61       206

    accuracy              

              precision    recall  f1-score   support

           0       0.94      0.98      0.96      1677
           1       0.72      0.51      0.60       206

    accuracy                           0.92      1883
   macro avg       0.83      0.74      0.78      1883
weighted avg       0.92      0.92      0.92      1883

Epoch: 40 | Epoch Time: 13m 10s
	Training Loss: 0.011564 \Test Loss: 0.028233
              precision    recall  f1-score   support

           0       0.98      0.87      0.92      1677
           1       0.44      0.86      0.58       206

    accuracy                           0.87      1883
   macro avg       0.71      0.86      0.75      1883
weighted avg       0.92      0.87      0.88      1883

Epoch: 41 | Epoch Time: 12m 29s
	Training Loss: 0.011372 \Test Loss: 0.018191
              precision    recall  f1-score   support

           0       0.99      0.74      0.85      1677
           1       0.31      0.93      0.46       206

    accuracy              